In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#read data
# test_set = np.loadtxt("mnist_test.csv",skiprows=1,delimiter=",")
# dataset = np.loadtxt("mnist_train.csv",skiprows=1,delimiter=",")

#
dataset = pd.read_csv("./mnist_train.csv", delimiter=',', nrows = 101)
# print(dataset.loc[0]) #Name: 0, Length: 785, dtype: int64
# print(dataset.loc[0][0]) #5
#input:dataset.loc[i][1:]
#truth:dataset.loc[i][0]

#can change to optimize
numLayer = 1
numNeu = 5
learn_rate = 0.1
epochs = 20
numWeight = 784*numNeu + numNeu*10 # only working when numLayer = 1
numOutputs = 10
numBias = numNeu + numOutputs

input_train = dataset[dataset.columns[1:]]
output_gt_val = dataset[dataset.columns[0]]

output_gt_set = np.zeros([100,10])
for i in range(100):
    for j in range(10):
        if j == output_gt_val[i]: 
            output_gt_set[i][j] = 1 
#print(output_gt_set)
def sigmoid(x):
  # Sigmoid activation function: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
  # Derivative of sigmoid: f'(x) = f(x) * (1 - f(x))
  fx = sigmoid(x)
  return fx * (1 - fx)

def mse_loss(y_true, y_pred):
  # y_true and y_pred are numpy arrays of the same length.
  return ((y_true - y_pred) ** 2).mean()

class NeuralNet():
    def __init__(self):
        self.weight = {}
        self.bias = {}
        self.loss = {}
        self.output = np.zeros([100,10])
        self.output_sum = np.zeros([100,10])
        self.layer1 = np.zeros(numNeu)
        self.sum_layer1 = np.zeros(numNeu)
        self.dyp_dweight = np.zeros(10*numNeu)
        self.dhl_dweight = np.zeros(784*numNeu)
        self.dhl_dbias = np.zeros(numNeu+numOutputs)
        self.dyp_dbias = np.zeros(numOutputs)
        self.yp_hl = np.zeros(numNeu)
        self.deriv_ypred_each = np.zeros(10)
        
    
    def init_weights_and_bias(self):
        for i in range(numWeight):
            self.weight[i] = np.random.normal()

        for i in range(numBias):
            self.bias[i] = np.random.normal()
    
    def test(self, input):
        output = np.zeros(10)
        index_weight = 0
        for j in range(784):
            for k in range(numNeu):
                self.sum_layer1[k] += input[j]*self.weight[index_weight] + self.bias[k]
                index_weight+=1
    
        # sigmoid layer
        for ii in range(numNeu):
            self.layer1[ii] = sigmoid(self.sum_layer1[ii])
            
        #hidden layer1 to output
        for ii in range(10):
            for j in range(5):
                output[ii] += self.layer1[j] * self.weight[index_weight] + self.bias[ii+5]
            output[ii] = sigmoid(output[ii])
        return output

    def forward(self,input,output_gt_set):
        
        for epoch in range(epochs):

            print(epoch)
            # input to hidden layer1
            for i in range(100): #100 data len
                i_input = input_train.loc[i]
                index_weight = 0
                for j in range(784):
                    for k in range(numNeu):
                        self.sum_layer1[k] += i_input[j]*self.weight[index_weight] + self.bias[k]
                        index_weight+=1
            
                # sigmoid layer
                for ii in range(numNeu):
                    self.layer1[ii] = sigmoid(self.sum_layer1[ii])
                    
                #hidden layer1 to output
                for ii in range(10):
                    for j in range(5):
                        self.output_sum[i][ii] += self.layer1[j] * self.weight[index_weight] + self.bias[ii+5]
                    self.output[i][ii] = sigmoid(self.output_sum[i][ii])
                
                # dl/dw = dl/dyp * dyp/dh (hidden layer) * dh/dw
                # dl/db = dl/dyp * dyp/dh (hidden layer) * dh/db
                #mse loss
                
                diff = (self.output[i] - output_gt_set[i])
                dL_dypred = 2 * np.sum(diff)
                for j in range(10):
                    self.deriv_ypred_each[j] = self.output[i][j] - output_gt_set[i][j]

                #所有weight的index+1是 % 5 == 1 ==>  h0
                #               % 5 == 2 ==>  h1
                #               % 5 == 3 ==>  h2
                #               % 5 == 4 ==>  h3
                #               % 5 == 0 ==>  h4
                #          一共784轮
                
                # deriv weight input-hidden
                weight_index_bp = 0
                for j in range(784):
                    for k in range(numNeu):
                        if (k+1) % 5 == 1:
                            self.dhl_dweight[weight_index_bp] = i_input[j] * deriv_sigmoid(self.sum_layer1[0])
                        elif (k+1) % 5 == 2:
                            self.dhl_dweight[weight_index_bp] = i_input[j] * deriv_sigmoid(self.sum_layer1[1])
                        elif (k+1) % 5 == 3:
                            self.dhl_dweight[weight_index_bp] = i_input[j] * deriv_sigmoid(self.sum_layer1[2])
                        elif (k+1) % 5 == 4:
                            self.dhl_dweight[weight_index_bp] = i_input[j] * deriv_sigmoid(self.sum_layer1[3])
                        elif (k+1) % 5 == 0:
                            self.dhl_dweight[weight_index_bp] = i_input[j] * deriv_sigmoid(self.sum_layer1[4])
                        weight_index_bp += 1

                # deriv weight hidden-output
                weight_index_bp = 0
                for j in range(10):
                    for k in range(numNeu):
                        self.dyp_dweight[weight_index_bp] = deriv_sigmoid(self.output[i][j]) * self.layer1[k]
                        weight_index_bp += 1
                
                # deriv hidden-output yp_hl 
                # numNeu = 5
                weight_index_bp = numNeu * 784
                for k in range(10):
                    for j in range(numNeu):
                        self.yp_hl[j] = self.weight[weight_index_bp] * deriv_sigmoid(self.output_sum[i][k])
                        weight_index_bp += 1


                
                # deriv bias
                for j in range(numNeu):
                    self.dhl_dbias[j] = deriv_sigmoid(self.sum_layer1[j])
                
                for j in range(numOutputs):
                    self.dhl_dbias[j+numNeu] = deriv_sigmoid(self.output_sum[i][j])



                # update each weight
                # self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                    # before hl
                #w0: in0->hl0
                #w1: in0->hl1
                #w2: in0->hl2
                ind = 0
                for j in range(784):
                    for k in range(numNeu):
                        self.weight[ind] -= learn_rate * dL_dypred *self.dhl_dweight[ind] * self.yp_hl[k]
                        ind += 1

                    # after hl
                    #self.w5 -= learn_rate * d_L_d_y1 * d_ypred_d_w5
                ind = 0
                for j in range(numNeu):
                    for k in range(10):
                        self.weight[ind+784*numNeu] -= learn_rate * self.deriv_ypred_each[k] * self.dyp_dweight[ind]
                        ind += 1

                # update each bias
                    #bias in hl self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2
                for j in range(numNeu):
                    self.bias[j] -= learn_rate * dL_dypred * self.yp_hl[j] * self.dhl_dbias[j]
                #bias in yp self.b3 -= learn_rate * d_L_d_y1 * d_ypred_d_b3
                for j in range(numOutputs):
                    self.bias[j+numNeu] -= learn_rate * self.deriv_ypred_each[j] * self.dhl_dbias[j+numNeu]

                # def mse_loss(y_true, y_pred):
            print(mse_loss(output_gt_set,self.output))




                    

                







                
            
            





aa = NeuralNet()
aa.init_weights_and_bias()
print("fin")
print(output_gt_val)

aa.forward(dataset.loc[0:][1:],output_gt_set)

print(aa.test(input_train.values.tolist()[0]), output_gt_val[0])

# 1. forward
# 2. loss calcualte
# 3. bp
# 4. repeat



fin
0      5
1      0
2      4
3      1
4      9
      ..
96     7
97     8
98     3
99     1
100    5
Name: label, Length: 101, dtype: int64
0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: overflow encountered in exp


0.18129326200299187
1
0.1773853293290173
2
0.17749907487793876
3
0.1776555601393452
4
0.17760390120781117
5
0.17730808347940416
6
0.17727355206529052
7
0.17812937691763897
8
0.17782849233064335
9
0.1774194875993675
10
0.17737796988826773
11
0.17744080074835683
12
0.1775739668446169
13
0.17779136634985696
14
0.1779668774751707
15
0.1780169554333213
16
0.1780166377889301
17
0.17804260395998328
18
0.17811503304349058
19
0.17822761163190295
[4.88412851e-01 6.14491272e-02 2.17026385e-03 3.48769339e-01
 1.25054909e-04 7.41689131e-01 1.62372341e-02 2.26056882e-01
 9.70628467e-02 9.99900549e-01] 5


In [2]:
print(aa.test(input_train.values.tolist()[0]), output_gt_val[0])

[0.47083978 0.76042087 0.00079617 0.42050592 0.04771674 0.67999765
 0.44681841 0.61897953 0.67143945 0.4582522 ] 5


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: overflow encountered in exp


In [3]:
print(aa.test(input_train.values.tolist()[1]), output_gt_val[1])

[0.47083978 0.76042087 0.00079617 0.42050592 0.04771674 0.67999765
 0.44681841 0.61897953 0.67143945 0.4582522 ] 0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: overflow encountered in exp


In [12]:
for i in range(100):
    #print(input_train.values.tolist()[i])
    print(np.argmax(aa.test(input_train.values.tolist()[i])))
    


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: overflow encountered in exp


9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
